# MFCC-Feature-Generierung

In [6]:
max_file_count = None
none_replacement = 'all'

SAMPLE_RATE = 22_050

# Daten einlesen

In [2]:
import os
import sys
sys.path.append(os.path.join((os.path.join(os.pardir, os.pardir)), os.pardir))
from helper.load_data import load_data

# determine path to `src/data`
cwd = os.getcwd()
src_dir = os.path.join(cwd, os.path.join(os.path.join(os.pardir, os.pardir), os.pardir))
data_dir = os.path.join(src_dir, 'data')

# load files
data_file_paths = load_data(data_dir)

In [3]:
max_file_count = min(max_file_count, len(data_file_paths)) if max_file_count is not None else None

## Audio-Daten einlesen

In [4]:
# do not print warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter('ignore')

In [5]:
import librosa

def get_feature(waveform, sample_rate):
    return librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=1)

def get_label(label_str):
    return -1 if (label_str == 'No_Whistle') else 1

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm

audio_data = []
labels = []

paths = data_file_paths[:max_file_count] if max_file_count is not None else data_file_paths

for file in tqdm(paths, unit='file'):
    base = os.path.splitext(file)[0]
    extension = os.path.splitext(file)[1]
    
    if extension == '.flac' and os.path.isfile(base + '.csv'):
        # print(os.path.basename(file))
        label_df = pd.read_csv(base + '.csv')

        for index, row in label_df.iterrows():
            start = max(0, row['start'])
            end = row['end']
            duration = end - start
            
            if duration <= 0:
                continue
            
            waveform, sample_rate = librosa.load(
                file,
                sr=SAMPLE_RATE,
                mono=True,
                offset=start,
                duration=duration)
            
            audio_data.append([waveform, sample_rate])
            
            label = get_label(row['label'])
            labels.append(label)

100%|██████████| 120/120 [01:22<00:00,  1.46file/s]


## Padding der Audio-Daten und MFCC-Generierung

In [8]:
max_length = 0
for file in audio_data:
    waveform = file[0]
    max_length = max(max_length, len(waveform))
print(max_length)

9064367


In [9]:
feature_matrix = []

for index, file in enumerate(tqdm(audio_data, unit='file')):
    waveform = file[0]
    sample_rate = file[1]
    
    padded_audio = waveform
    if len(waveform) < max_length:
        padded_audio = np.pad(waveform, (0, max_length - len(waveform)), mode='constant')
    
    feature = get_feature(padded_audio, sample_rate)
    feature_matrix.append(feature)

100%|██████████| 1007/1007 [08:30<00:00,  1.97file/s]


In [10]:
feature_matrix = np.array(feature_matrix).squeeze()
feature_matrix = feature_matrix.reshape(feature_matrix.shape[0], -1)

labels = np.array(labels)

In [11]:
np.save(f'feature_{max_file_count or none_replacement}.npy', feature_matrix)
np.save(f'labels_{max_file_count  or none_replacement}.npy', labels)

In [12]:
print(feature_matrix.shape)
print(labels.shape)

(1007, 17704)
(1007,)
